### Урок 2. Непрерывность, гладкость и сходимость ЦФ. Дискретные ЦФ#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import numpy as np    
import math
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

### Задание 1.

Директор выделил машину под закупку оборудования, выделил достаточно большое количество денег и сказал: «Берите, что вам нужно, но не более 200 кг. И каждого товара берите не более одной единицы!» Есть прайс-лист на 2000 наименований.  
Стоимости товаров варьируются от \\$100 до \\$5000 с шагом \\$100.  
Массы товаров варьируются от 1 до 150 кг с шагом в 1 кг.  
Зависимостей между массой и стоимостью нет (может выпасть товар массой 1 кг и стоимостью \\$5000 и массой 150 кг и стоимостью \\$100.  
1. Необходимо составить ЦФ для этой задачи, выбрать критерий оптимальности и...  
2. *...предложить алгоритм её решения.  

Предположим, что нам нужно набрать оборудовани на 200кг (или менее) по минимальной цене, т.е. мин \\$/кг.
Т.е. нас интересуют тяжелые и дешевые.  
Целевая функция:  
W(i) - все комбинации оборудования дающие 200 кг  
P(i) - цена комбинации оборудования  
F - целевая функция (цена в \\$)  

#### Критерий оптимальности:  F = min ( W(i) * P(i) )

In [2]:
np.random.seed(2177)

In [3]:
# Прайс лист из 2000 наименований
# x[0] - Номер оборудования
# x[1] - вес
# x[2] - цена
Price_list = [[x, np.random.randint(1, 150), np.random.randint(1, 50)*100] for x in range(0, 2000)]

In [4]:
# Прайс лист, но со стоимостью за кг
# x[3] - $/кг
Price_per_kg = [[x[0], x[1], x[2], float(x[2]/x[1])] for x in Price_list]

In [5]:
# Сортируем прайс по стоимости за кг
Price_per_kg.sort(key=lambda i: i[3])

In [6]:
Price_per_kg[0:10]

[[1020, 146, 100, 0.684931506849315],
 [1382, 144, 100, 0.6944444444444444],
 [1555, 141, 100, 0.7092198581560284],
 [1723, 140, 100, 0.7142857142857143],
 [392, 138, 100, 0.7246376811594203],
 [79, 136, 100, 0.7352941176470589],
 [1268, 133, 100, 0.7518796992481203],
 [1802, 128, 100, 0.78125],
 [171, 126, 100, 0.7936507936507936],
 [1199, 118, 100, 0.847457627118644]]

In [7]:
min_price = 5000
list_of_obj = []  # Список номеров оборудования с мин ценой
for st in range(0, 2000): # Берем все элементы по очереди
    weight = 0
    price = 0
    current_list = []
    for i in Price_per_kg[st:]: # Перебираем все элементы от текущего
        if min_price < i[2]: # Если стоимость оборудование больше, чем цена уже имеющегося набора, то все останавливаем
            break
        if (weight + i[1]) <= 200: # Набираем элементы до 200 кг по увеличению стоимости за кг
            weight += i[1]
            price += i[2]
            current_list.append(i[0]) # Добавляем номера подходящего оборудования в список
        if weight == 200:
            if price < min_price:  # Если полученая цена меньше чем у предыдущего набора, то записываем ее и новый список
                min_price = price
                list_of_obj = current_list
            continue            

In [8]:
list_of_obj

[1723, 1046]

In [9]:
result = [x for x in Price_per_kg if (x[0] in list_of_obj)]

In [10]:
result

[[1723, 140, 100, 0.7142857142857143], [1046, 60, 100, 1.6666666666666667]]

#### Ответ: Получаем список оборудования для покупки (на np.random.seed(2177)) на 200 кг (140+60), стоимостью 200\\$ (100+100) и номерами по каталогу 1723 и 1046

### Задание 2.

Предприятие выпускает покрышки и надувные лодки.  
Производство одной покрышки занимает 2 часа на заготовительном участке, 4 часа на участке обработки, 0 часов на участке сборки.  
Производство одной лодки занимает 6 часов на заготовительном участке, 3 часа на участке обработки, 2 часа на участке сборки.  
Стоимость одной лодки — 12000 рублей, стоимость покрышки — 7000 рублей.  
Фонд времени в день: заготовительного участка — 14 нормочасов, участка обработки — 10 нч, участка сборки — 8 нч.  
1. Составить ЦФ, записать ограничения и функцию Лагранжа для решения этой задачи.  
* Разработать оптимальный производственный план предприятия.  

$F = 7000 * n1 + 12000 * n2$  
$F(n1, n2) = max(F)$  

### Ограничения:

$2 * n1 + 6 * n2 \leq 14$  
$4 * n1 + 3 * n2 \leq 10$  
$0 * n1 + 2 * n2 \leq 8$

### Функция Лагранжа:

$L(n_1,n_2,\lambda_1,\lambda_2,\lambda_3) = 7000n_1+12000n_2 + \lambda_1(2n_1+6n_2-14) + \lambda_2(4n_1+3n_2-10) + \lambda_3(2n_2-8)$<br>

т.к. условие по $\lambda_3(2n_2-8)$ недостижимо (на заготовительном участке потребуется 6*4=24 часа), то $\lambda_3$ уберем

$L(n_1,n_2,\lambda_1,\lambda_2) = 7000n_1+12000n_2 + \lambda_1(2n_1+6n_2-14) + \lambda_2(4n_1+3n_2-10)$<br>

Запишем уравнения для частных производных функции Лагранжа:<br>
$\frac{\delta{L(n_1,n_2,\lambda_1,\lambda_2)}}{\delta{n_1}} = 7000 + 2\lambda_1 + 4\lambda_2 = 0$<br>
$\frac{\delta{L(n_1,n_2,\lambda_1,\lambda_2)}}{\delta{n_2}} = 12000 + 6\lambda_1 + 3\lambda_2 = 0$<br>
$\frac{\delta{L(n_1,n_2,\lambda_1,\lambda_2)}}{\delta{\lambda_1}} = 2n_1+6n_2-14 = 0$<br>
$\frac{\delta{L(n_1,n_2,\lambda_1,\lambda_2)}}{\delta{\lambda_2}} = 4n_1+3n_2-10 = 0$<br>

Для этой задачи подмножество двух последних уравнений будет независимым от первых двух и совместным относительно $n_1$ и $n_2$.  
Решая систему, получим $n_1^* = 1$, $n_2^* = 2$.<br>
Ответ: наибольшую прибыль в размере 31000 рублей передприятие получит, производя 1 покрышку и 2 лодки в день.